In [ ]:
%pip install pandas emoji openai tiktoken sklearn

## Import the requisite libraries

In [1]:
# Importing libraries
import pandas as pd
import emoji
import re
import string
import json

# Loading data from CSV files
test_data = pd.read_csv("./data/issues/issues_test.csv")
train_data = pd.read_csv("./data/issues/issues_train.csv")

## Data Visualization

In [2]:
test_data

,repo,created_at,label,title,body
0,facebook/react,2023-08-02 02:26:00,bug,Bug: [18.3.0-canary] renderToString hoists som...,<!--\r\n Please provide a clear and concise d...
1,facebook/react,2023-07-17 22:43:05,bug,[DevTools Bug]: Chrome extension gets disconne...,### Website or app\r\n\r\nhttps://react.dev/\r...
2,facebook/react,2023-07-13 19:01:47,bug,[DevTools Bug]: Deprecated __REACT_DEVTOOLS_GL...,### Website or app\r\n\r\nN/A\r\n\r\n### Repro...
3,facebook/react,2023-06-07 17:26:43,bug,[DevTools Bug]: React devtools stuck at Loadin...,### Website or app\r\n\r\ncorporate project (p...
4,facebook/react,2023-05-31 15:17:41,bug,Bug: Radio button onChange not called in curre...,<!--\r\n Please provide a clear and concise d...
...,...,...,...,...,...
1495,opencv/opencv,2022-01-22 11:52:21,feature,Task: GCC 12 support,Support compilation with GCC 12 and fix tests\...
1496,opencv/opencv,2022-01-16 19:27:55,feature,AudioIO: add dnn speech recognition sample on C++,### Pull Request Readiness Checklist\r\n\r\nSe...
1497,opencv/opencv,2022-01-14 22:05:58,feature,Use modern OpenVINO package interface,"* new cmake options: `WITH_OPENVINO`, `OPENCV_..."
1498,opencv/opencv,2022-01-12 09:14:41,feature,TiffEncoder write support more depth type,**Merge with extra**: https://github.com/openc...


In [34]:
train_data

,repo,created_at,label,title,body
0,facebook/react,2023-08-26 06:33:37,bug,"[DevTools Bug] Cannot add node ""1"" because a n...",### Website or app\n\nPrivate repo cannot give...
1,facebook/react,2023-07-28 05:16:12,bug,[DevTools Bug]: Devtools extension build faili...,### Website or app\n\nN/A\n\n### Repro steps\n...
2,facebook/react,2023-07-13 21:58:31,bug,[DevTools Bug]: Deprecated __REACT_DEVTOOLS_GL...,### Website or app\n\nhttps://github.com/open-...
3,facebook/react,2023-06-14 02:31:20,bug,"[DevTools Bug] Cannot remove node ""0"" because ...",### Website or app\n\nlocal\n\n### Repro steps...
4,facebook/react,2023-06-03 11:29:44,bug,"[DevTools Bug] Cannot remove node ""103"" becaus...",### Website or app\n\nlocalhost\n\n### Repro s...
...,...,...,...,...,...
1495,opencv/opencv,2022-01-24 10:48:13,feature,core: FP denormals support,relates #21046\r\n\r\n- support x86 SSE FTZ+DA...
1496,opencv/opencv,2022-01-20 12:40:55,feature,feature: submodule or a class scope for export...,All classes are registered in the scope that c...
1497,opencv/opencv,2022-01-15 02:39:22,feature,Reading BigTiff images,**Merge with extra: https://github.com/opencv/...
1498,opencv/opencv,2022-01-14 15:37:53,feature,Add general broadcasting layer,Performance details(broadcasting 1x1 to 16x204...


## Data Preprocessing
### Data Cleaning: Method 1  
Within this notebook, we employ two distinct data cleaning methodologies. This tailored approach is followed given variations among the repositories, each showing a more favorable outcome in response to one or the other cleaning methods.

In [3]:
# Initialize counters for text cleaning
cleaned_count = 0
original_count = 0

# Text cleaning function
def clean_text(text):
    global cleaned_count, original_count

    if not isinstance(text, str):
        original_count += 1
        return text

    # Remove double quotation marks
    text = text.replace('"', '')

    # Remove text starting with "DevTools" and ending with "(automated)"
    text = re.sub(r'DevTools.*?\(automated\)', '', text)

    # Lowercasing should be one of the first steps to ensure uniformity
    text = text.lower()

    # Remove emojis
    text = emoji.demojize(text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters and punctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)

    # Remove '#' characters
    text = text.replace("#", "")

    # Remove consecutive whitespaces and replace with a single space
    text = re.sub(r'\s+', ' ', text)

    # Split the text into words
    words = text.split()

    # Remove words that are over 20 characters
    words = [word for word in words if len(word) <= 20]

    # Join the remaining words back into cleaned text
    cleaned_text = ' '.join(words)

    cleaned_count += 1
    return cleaned_text

test_data['body'] = test_data['body'].apply(clean_text)
test_data['title'] = test_data['title'].apply(clean_text)


print(f"Cleaned {cleaned_count} times.")
print(f"Returned original text {original_count} times.")

train_data['body'] = train_data['body'].apply(clean_text)
train_data['title'] = train_data['title'].apply(clean_text)


print(f"Cleaned {cleaned_count} times.")
print(f"Returned original text {original_count} times.")

Cleaned 2998 times.
Returned original text 2 times.
Cleaned 5998 times.
Returned original text 2 times.


## Data Division  

Subsequently, we partitioned our dataset into five smaller dataframes, ensuring an exclusive handling of each project. This segregation was executed on both the training and testing datasets.

In [4]:
test_data_facebook = test_data[: 300]
test_data_tensorflow = test_data[300: 600]
test_data_microsoft = test_data[600: 900]
test_data_bitcoin = test_data[900: 1200]
test_data_opencv= test_data[1200: 1500]

In [5]:
train_data_facebook = train_data[: 300]
train_data_tensorflow = train_data[300: 600]
train_data_microsoft = train_data[600: 900]
train_data_bitcoin = train_data[900: 1200]
train_data_opencv= train_data[1200: 1500]

## Fine-Tuning  
We fine-tuned ChatGPT-3.5-Turbo using the training data, aiming to achieve superior performance compared to the standard approach of invoking the OpenAI API GPT-4 model.

In [3]:
# Invoking the API
from openai import OpenAI
client = OpenAI(api_key = 'open-ai-api')

## Facebook repository dataset fine-tuning process
### Data Transformation  
Prior to beginning the fine-tuning process, our initial step involves transforming our dataframe into a JSON line file format. This formatted file will serve as the prompt input for the fine-tuning process. Each prompt will encapsulate the title and body details of every pull request. Our anticipated outcome from the fine-tuned model will be the corresponding label for each PR, distinguishing between bug reports, questions, or feature requests.

In [39]:
# Open the file in write mode
with open('data/conversationaldata/conversational_data_facebook.jsonl', 'w', encoding='utf-8') as f:
    # Iterate over the rows in the DataFrame
    for index, row in train_data_facebook.iterrows():
        # Create the user message by formatting the prompt with the title and body
        user_message = f"Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:\n{row['title']}\n{row['body']}"
        
        # Create the assistant message by taking the label
        assistant_message = row['label']
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "GitHub Issue Report Classifier"},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

## Training file  
With our JSON line file generated, it now serves as the foundational conversation input for our fine-tuned model. We're prepared to upload this training file to the OpenAI API to initiate the training process.

In [40]:
## Uplopading a training file
ft_file_fb = client.files.create(
  file=open("data/conversationaldata/conversational_data_facebook.jsonl", "rb"),
  purpose="fine-tune"
)
ft_file_fb

FileObject(id='file-0Aub147vzoJf7tQGxAYXo2eF', bytes=363360, created_at=1701751545, filename='conversational_data_facebook.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

## Model creation  
At last, the stage is set to create the model, designated with the suffix 'repo-prissueclassifier.'

In [41]:
## Creating a fine-tuned model
ft_job_fb = client.fine_tuning.jobs.create(
  training_file=ft_file_fb.id, 
  model="gpt-3.5-turbo",
  suffix= "fb-issueclassifier"
)

ft_job_fb

FineTuningJob(id='ftjob-LG8EargXmVTFrGEWBYmF841X', created_at=1701751546, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-CpaRU3Zq9ePCCtbhezmcbgrg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-Mz8izUzswJ5M3H1C8S2aO6X0', validation_file=None)

In [38]:
# Retrieving the state of a fine-tune
facebook_ft_model = client.fine_tuning.jobs.retrieve(ft_job_fb.id).fine_tuned_model
print(facebook_ft_model)

ft:gpt-3.5-turbo-0613:gcucst440:fb-issueclassifier:8LLGMnAI


In [ ]:
# You can track the progress of your fine-tuning job by listing the lastest events. On our models it took about 3 hours to fine-tune each model
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_fb.id, limit=20)

## Tensorflow repository dataset fine-tuning process

In [44]:
# Open the file in write mode
with open('data/conversationaldata/conversational_data_tensorflow.jsonl', 'w', encoding='utf-8') as f:
    # Iterate over the rows in the DataFrame
    for index, row in train_data_tensorflow.iterrows():
        # Create the user message by formatting the prompt with the title and body
        user_message = f"Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:\n{row['title']}\n{row['body']}"
        
        # Create the assistant message by taking the label
        assistant_message = row['label']
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "GitHub Issue Report Classifier"},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

In [45]:
ft_file_tf = client.files.create(
  file=open("data/conversationaldata/conversational_data_tensorflow.jsonl", "rb"),
  purpose="fine-tune"
)
ft_file_tf.id

FileObject(id='file-otqI5x1eZvI9ifBIdhE1buie', bytes=551677, created_at=1701751548, filename='conversational_data_tensorflow.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [46]:
## Creating a fine-tuned model
ft_job_tf = client.fine_tuning.jobs.create(
  training_file=ft_file_tf.id, 
  model="gpt-3.5-turbo",
  suffix= "tf-issueclassifier"
)
ft_job_tf

FineTuningJob(id='ftjob-JPF1YSBlCiLAdOHc3egX9XjI', created_at=1701751548, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-CpaRU3Zq9ePCCtbhezmcbgrg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-poTXDVFaDaPWgI4OWkzKtHCY', validation_file=None)

In [47]:
# Retrieving the state of a fine-tune
tensorflow_ft_model = client.fine_tuning.jobs.retrieve(ft_job_tf.id).fine_tuned_model
print(tensorflow_ft_model)

ft:gpt-3.5-turbo-0613:gcucst440:tf-issueclassifier:8LLGZuRu


In [ ]:
# You can track the progress of your fine-tuning job by listing the lastest events. On our models it took about 3 hours to fine-tune each model
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_tf.id, limit=20)

## Microsoft repository dataset fine-tuning process

In [49]:
# Open the file in write mode
with open('data/conversationaldata/conversational_data_microsoft.jsonl', 'w', encoding='utf-8') as f:
    # Iterate over the rows in the DataFrame
    for index, row in train_data_microsoft.iterrows():
        # Create the user message by formatting the prompt with the title and body
        user_message = f"Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:\n{row['title']}\n{row['body']}"
        
        # Create the assistant message by taking the label
        assistant_message = row['label']
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "GitHub Issue Report Classifier"},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

In [50]:
ft_file_ms = client.files.create(
  file=open("data/conversationaldata/conversational_data_microsoft.jsonl", "rb"),
  purpose="fine-tune"
)
ft_file_ms

FileObject(id='file-SPe4gJF5xHVBw1bfley7BddS', bytes=300117, created_at=1701751551, filename='conversational_data_microsoft.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [21]:
## Creating a fine-tuned model
ft_job_ms = client.fine_tuning.jobs.create(
  training_file=ft_file_ms.id, 
  model="gpt-3.5-turbo",
  suffix= "ms-issueclassifier"
)
ft_job_ms

FineTuningJob(id='ftjob-fDSapt9Eh3trHScN7SOW5wIn', created_at=1701829792, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-CpaRU3Zq9ePCCtbhezmcbgrg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-o2pr8FckpXuZhncbVF9Fykzl', validation_file=None)

In [23]:
# Retrieving the state of a fine-tune
microsoft_ft_model = client.fine_tuning.jobs.retrieve(ft_job_ms.id).fine_tuned_model
print(microsoft_ft_model)

ft:gpt-3.5-turbo-0613:gcucst440:ms-issueclassifier:8LLFl5QI


In [ ]:
# You can track the progress of your fine-tuning job by listing the lastest events. On our models it took about 3 hours to fine-tune each model
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_ms.id, limit=20)

## Bitcoin repository dataset fine-tuning process

In [ ]:
# Open the file in write mode
with open('data/conversationaldata/conversational_data_bitcoin.jsonl', 'w', encoding='utf-8') as f:
    # Iterate over the rows in the DataFrame
    for index, row in train_data_bitcoin.iterrows():
        # Create the user message by formatting the prompt with the title and body
        user_message = f"Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:\n{row['title']}\n{row['body']}"
        
        # Create the assistant message by taking the label
        assistant_message = row['label']
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "GitHub Issue Report Classifier"},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

In [ ]:
ft_file_bc = client.files.create(
  file=open("data/conversationaldata/conversational_data_bitcoin.jsonl", "rb"),
  purpose="fine-tune"
)
ft_file_bc

FileObject(id='file-jDVrdHpKeEy23ttxa5KCfk7O', bytes=548310, created_at=1700103758, filename='conversational_data_bitcoin.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
## Creating a fine-tuned model
ft_job_bc = client.fine_tuning.jobs.create(
  training_file=ft_file_bc.id, 
  model="gpt-3.5-turbo",
  suffix= "bc-issueclassifier"
)
ft_job_bc

FineTuningJob(id='ftjob-dBpnVHyaN48iF0SC5QMwzCCn', created_at=1700103775, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-CpaRU3Zq9ePCCtbhezmcbgrg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-jDVrdHpKeEy23ttxa5KCfk7O', validation_file=None)

In [ ]:
# Retrieving the state of a fine-tune
bitcoin_ft_model = client.fine_tuning.jobs.retrieve(ft_job_bc.id).fine_tuned_model
print(bitcoin_ft_model)

ft:gpt-3.5-turbo-0613:gcucst440:bc-issueclassifier:8LOOptG5


In [ ]:
# You can track the progress of your fine-tuning job by listing the lastest events. On our models it took about 3 hours to fine-tune each model
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_bc.id, limit=20)

## OpenCV repository dataset fine-tuning process

In [ ]:
# Open the file in write mode
with open('data/conversationaldata/conversational_data_opencv.jsonl', 'w', encoding='utf-8') as f:
    # Iterate over the rows in the DataFrame
    for index, row in train_data_opencv.iterrows():
        # Create the user message by formatting the prompt with the title and body
        user_message = f"Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:\n{row['title']}\n{row['body']}"
        
        # Create the assistant message by taking the label
        assistant_message = row['label']
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "GitHub Issue Report Classifier"},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

In [ ]:
ft_file_oc = client.files.create(
  file=open("data/conversationaldata/conversational_data_opencv.jsonl", "rb"),
  purpose="fine-tune"
)
ft_file_oc.id

FileObject(id='file-31FW3RwGmDhMZnIEtc95Oj5t', bytes=621942, created_at=1700104001, filename='conversational_data_opencv.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
## Creating a fine-tuned model
ft_job_oc = client.fine_tuning.jobs.create(
  training_file=ft_file_oc.id, 
  model="gpt-3.5-turbo",
  suffix= "cv-issueclassifier"
)
ft_job_oc.id

FineTuningJob(id='ftjob-XdaiNJQtq10dbFhMPdDfKk4t', created_at=1700104041, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-CpaRU3Zq9ePCCtbhezmcbgrg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-31FW3RwGmDhMZnIEtc95Oj5t', validation_file=None)

In [ ]:
# Retrieving the state of a fine-tune
opencv_ft_model = client.fine_tuning.jobs.retrieve(ft_job_oc.id).fine_tuned_model
print(opencv_ft_model)

ft:gpt-3.5-turbo-0613:gcucst440:cv-issueclassifier:8LOdV6zV


In [ ]:
# You can track the progress of your fine-tuning job by listing the lastest events. On our models it took about 3 hours to fine-tune each model
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_oc.id, limit=20)

## Fine-tuning results  
The successful fine-tuning of all models was completed using the default of 3 epochs. The process spanned approximately 5 hours; however, variations in processing time might occur due to queue dynamics at any given moment.

## Utilizing Fine-tuned model  
Next, another API from OpenAI is used to invoke the fine-tuned model and assess its performance on the testing dataset.

In [14]:
import openai
import time
import pandas as pd
import re
import concurrent.futures
import tiktoken
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Replace 'open-ai-key' with your actual OpenAI API key
openai.api_key = 'open-ai-api'

# max_token here should be one since 'bug', 'feature', and 'question' are one token long. This might change for future versions of the model and api but you can check the value on the
def query_chatgpt(prompt, model, temperature=0.0,  max_tokens=1, max_retries=5):
    """
    Function to query ChatGPT-4 with a given prompt, with retries for timeouts.

    :param prompt: Prompt string to send to ChatGPT-2.5
    :param model: The model to use, default is ChatGPT-3.5
    :param max_tokens: Maximum number of tokens to generate
    :param max_retries: Maximum number of retries for timeout
    :return: Response from ChatGPT-3.5 or None if all retries fail
    """
    attempt = 0
    max_content_tokens = 3999
    encoding = tiktoken.get_encoding("cl100k_base")
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Function to truncate the message and avoid passing the limit of 4k tokens per gpt-3.5 fine-tuned model limitations
    def truncate_message(message, max_length):
        tokens = encoding.encode(message)
        if len(tokens) > max_length:
            truncated_tokens = tokens[:max_length]
            message = encoding.decode(truncated_tokens)
        return message

    # Truncate the prompt if necessary
    prompt = truncate_message(prompt, max_content_tokens)

    while attempt < max_retries:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(
                openai.chat.completions.create,
                model=model,
                messages=[{"role": "system", "content": "GitHub Issue Report Classifier"}, {"role": "user", "content": prompt}],
                max_tokens=max_tokens,
                temperature=temperature
            )
            try:
                response = future.result(timeout=5)  # 5 seconds timeout
                return response.choices[0].message.content
            except concurrent.futures.TimeoutError:
                print(f"Attempt {attempt + 1}/{max_retries} - Request timed out. Retrying...")
            except Exception as e:
                print(f"Attempt {attempt + 1}/{max_retries} - An error occurred: {e}")
            finally:
                attempt += 1

    print("Failed to get a response after several retries.")
    return None
    
labels = ['feature', 'bug', 'question']

## Facebook React Repo Testing  
The function defined above is being called, passing the specific model for each repository and testing it with the testing dataset. It's essential to note the setup of a timer to comply with the "token per minute" limitations on the API. Additionally, the results of each iteration are printed for tracking and improvement purposes.

In [ ]:
y_true_fb = []
y_pred_fb = []

iterations = len(test_data_facebook)

# Now let's loop through the test data and classify the GitHub issues
for i in range(iterations):
    correct_label = test_data_facebook.iloc[i]['label'].lower()
    description = f"{test_data_facebook.iloc[i]['title']} \n {test_data_facebook.iloc[i]['body']}"
    print(f"Correct PR type: {correct_label}")
    
    prompt = f"Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:\n{description}"
    response = query_chatgpt(prompt, facebook_ft_model)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = re.sub(r'[^A-Za-z]+', '', response).lower().strip()
    print(f"Predicted PR type: {predicted_label}")
    
    # Append to lists for evaluation
    y_true_fb.append(correct_label)
    y_pred_fb.append(predicted_label)
    time.sleep(6)  # Wait for 6 seconds before retrying

# See output on outputs/cell51output.txt

## Calculating the results  
Once all testing data undergoes evaluation using the corresponding fine-tuned models, we'll leverage the two arrays generated—representing the true labels and predicted labels—to conduct result assessments.

For tracking purposes, a CSV file has been generated for each result.

In [ ]:
# Calculate weighted average F1-score, precision, and recall
f1_fb = f1_score(y_true_fb, y_pred_fb, labels=labels, average='weighted')
precision_fb = precision_score(y_true_fb, y_pred_fb, labels=labels, average='weighted')
recall_fb = recall_score(y_true_fb, y_pred_fb, labels=labels, average='weighted')

# Calculate confusion matrix
cm_fb = confusion_matrix(y_true_fb, y_pred_fb, labels=labels)

cm_df_fb = pd.DataFrame(cm_fb, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results_fb = {}
for i, label in enumerate(labels):
    results_fb[label] = {'TP': cm_fb[i, i]}
    results_fb[label]['FP'] = cm_fb[:, i].sum() - cm_fb[i, i]
    results_fb[label]['FN'] = cm_fb[i, :].sum() - cm_fb[i, i]
    results_fb[label]['TN'] = cm_fb.sum() - (results_fb[label]['TP'] + results_fb[label]['FP'] + results_fb[label]['FN'])

# Print results_fb
for label, metrics in results_fb.items():
    print(f"{label}: {metrics}")

# Save results_fb to CSV
results_fb_df = pd.DataFrame(results_fb).T
results_fb_df['F1-score'] = f1_fb
results_fb_df['Recall'] = recall_fb
results_fb_df['Precision'] = precision_fb

results_fb_df.to_csv('metrics/confusion_matrix_fb.csv')

print(f"Precision = {precision_fb}")
print(f"Recall = {recall_fb}")
print(f"F1-score = {f1_fb}")

feature: {'TP': 89, 'FP': 15, 'FN': 11, 'TN': 185}
bug: {'TP': 95, 'FP': 19, 'FN': 5, 'TN': 181}
question: {'TP': 74, 'FP': 8, 'FN': 26, 'TN': 192}
Precision = 0.8638471961642693
Recall = 0.86
F1-score = 0.8578621000281252


### F1-Score: 85.79%

## Evaluating the metrics  
Below, the metrics for each label are presented to facilitate a more precise evaluation.

In [ ]:
# Create a classification report
report_fb = classification_report(y_true_fb, y_pred_fb, labels=['bug', 'feature', 'question'], target_names=['bug', 'feature', 'question'], zero_division=0, output_dict=True)

# Convert the report to a DataFrame
report_df_fb = pd.DataFrame(report_fb).transpose()

# Print the classification report
print(report_df_fb)

              precision  recall  f1-score  support
bug            0.833333    0.95  0.887850   100.00
feature        0.855769    0.89  0.872549   100.00
question       0.902439    0.74  0.813187   100.00
accuracy       0.860000    0.86  0.860000     0.86
macro avg      0.863847    0.86  0.857862   300.00
weighted avg   0.863847    0.86  0.857862   300.00


## Tensorflow Repo Testing

In [ ]:
y_true_tf = []
y_pred_tf = []

iterations = len(test_data_tensorflow)

# Now let's loop through the test data and classify the GitHub pull requests
for i in range(iterations):
    correct_label = test_data_tensorflow.iloc[i]['label'].lower()
    description = f"{test_data_tensorflow.iloc[i]['title']} \n {test_data_tensorflow.iloc[i]['body']}"
    print(f"Correct PR type: {correct_label}")
    
    prompt = f"Classify, IN ONLY 1 WORD, the following GitHub pull request as 'feature', 'bug', or 'question' based on its title and body:\n{description}"
    response = query_chatgpt(prompt, tensorflow_ft_model)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = re.sub(r'[^A-Za-z]+', '', response).lower().strip()
    print(f"Predicted PR type: {predicted_label}")
    
    # Append to lists for evaluation
    y_true_tf.append(correct_label)
    y_pred_tf.append(predicted_label)
    time.sleep(6)  # Wait for 6 seconds before retrying

# See output on outputs/cell58output.txt

In [ ]:
# Calculate weighted average F1-score, precision, and recall
f1_tf = f1_score(y_true_tf, y_pred_tf, labels=labels, average='weighted')
precision_tf = precision_score(y_true_tf, y_pred_tf, labels=labels, average='weighted')
recall_tf = recall_score(y_true_tf, y_pred_tf, labels=labels, average='weighted')

# Calculate confusion matrix
cm_tf = confusion_matrix(y_true_tf, y_pred_tf, labels=labels)

cm_df_tf = pd.DataFrame(cm_tf, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results_tf = {}
for i, label in enumerate(labels):
    results_tf[label] = {'TP': cm_tf[i, i]}
    results_tf[label]['FP'] = cm_tf[:, i].sum() - cm_tf[i, i]
    results_tf[label]['FN'] = cm_tf[i, :].sum() - cm_tf[i, i]
    results_tf[label]['TN'] = cm_tf.sum() - (results_tf[label]['TP'] + results_tf[label]['FP'] + results_tf[label]['FN'])

# Print results_tf
for label, metrics in results_tf.items():
    print(f"{label}: {metrics}")

# Save results_tf to CSV
results_tf_df = pd.DataFrame(results_tf).T
results_tf_df['F1-score'] = f1_tf
results_tf_df['Recall'] = recall_tf
results_tf_df['Precision'] = precision_tf

results_tf_df.to_csv('metrics/confusion_matrix_tf.csv')

print(f"Precision = {precision_tf}")
print(f"Recall = {recall_tf}")
print(f"F1-score = {f1_tf}")

feature: {'TP': 81, 'FP': 6, 'FN': 19, 'TN': 194}
bug: {'TP': 88, 'FP': 8, 'FN': 12, 'TN': 192}
question: {'TP': 89, 'FP': 28, 'FN': 11, 'TN': 172}
Precision = 0.869461636703016
Recall = 0.86
F1-score = 0.861515280599043


### F1-score: 86.15%

In [ ]:
# Create a classification report
report_tf = classification_report(y_true_tf, y_pred_tf, labels=['bug', 'feature', 'question'], target_names=['bug', 'feature', 'question'], zero_division=0, output_dict=True)

# Convert the report to a DataFrame
report_df_tf = pd.DataFrame(report_tf).transpose()

# Print the classification report
print(report_df_tf)

              precision  recall  f1-score  support
bug            0.916667    0.88  0.897959   100.00
feature        0.931034    0.81  0.866310   100.00
question       0.760684    0.89  0.820276   100.00
accuracy       0.860000    0.86  0.860000     0.86
macro avg      0.869462    0.86  0.861515   300.00
weighted avg   0.869462    0.86  0.861515   300.00


## Microsoft Repo Testing

In [ ]:
y_true_ms = []
y_pred_ms = []

iterations = len(test_data_microsoft)

# Now let's loop through the test data and classify the GitHub pull requests
for i in range(iterations):
    correct_label = test_data_microsoft.iloc[i]['label'].lower()
    description = f"{test_data_microsoft.iloc[i]['title']}\n{test_data_microsoft.iloc[i]['body']}"
    print(f"Correct PR type: {correct_label}")
    
    prompt = f"Classify, IN ONLY 1 WORD, the following GitHub pull request as 'feature', 'bug', or 'question' based on its title and body:\n{description}"
    response = query_chatgpt(prompt, microsoft_ft_model)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = re.sub(r'[^A-Za-z]+', '', response).lower().strip()
    print(f"Predicted PR type: {predicted_label}")
    
    # Append to lists for evaluation
    y_true_ms.append(correct_label)
    y_pred_ms.append(predicted_label)
    time.sleep(6)  # Wait for 6 seconds before retrying

# See output on outputs/cell63output.txt

In [ ]:
# Calculate weighted average F1-score, precision, and recall
f1_ms = f1_score(y_true_ms, y_pred_ms, labels=labels, average='weighted')
precision_ms = precision_score(y_true_ms, y_pred_ms, labels=labels, average='weighted')
recall_ms = recall_score(y_true_ms, y_pred_ms, labels=labels, average='weighted')

# Calculate confusion matrix
cm_ms = confusion_matrix(y_true_ms, y_pred_ms, labels=labels)

cm_df_ms = pd.DataFrame(cm_ms, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results_ms = {}
for i, label in enumerate(labels):
    results_ms[label] = {'TP': cm_ms[i, i]}
    results_ms[label]['FP'] = cm_ms[:, i].sum() - cm_ms[i, i]
    results_ms[label]['FN'] = cm_ms[i, :].sum() - cm_ms[i, i]
    results_ms[label]['TN'] = cm_ms.sum() - (results_ms[label]['TP'] + results_ms[label]['FP'] + results_ms[label]['FN'])

# Print results_ms
for label, metrics in results_ms.items():
    print(f"{label}: {metrics}")

# Save results_ms to CSV
results_ms_df = pd.DataFrame(results_ms).T
results_ms_df['F1-score'] = f1_ms
results_ms_df['Recall'] = recall_ms
results_ms_df['Precision'] = precision_ms


results_ms_df.to_csv('metrics/confusion_matrix_ms.csv')

print(f"Precision = {precision_ms}")
print(f"Recall = {recall_ms}")
print(f"F1-score = {f1_ms}")

feature: {'TP': 84, 'FP': 21, 'FN': 16, 'TN': 179}
bug: {'TP': 76, 'FP': 13, 'FN': 24, 'TN': 187}
question: {'TP': 80, 'FP': 26, 'FN': 20, 'TN': 174}
Precision = 0.802883188467246
Recall = 0.8
F1-score = 0.8001480094936562


### F1-score: 80.01%

In [ ]:
# Create a classification report
report_ms = classification_report(y_true_ms, y_pred_ms, labels=['bug', 'feature', 'question'], target_names=['bug', 'feature', 'question'], zero_division=0, output_dict=True)

# Convert the report to a DataFrame
report_df_ms = pd.DataFrame(report_ms).transpose()

# Print the classification report
print(report_df_ms)

              precision  recall  f1-score  support
bug            0.853933    0.76  0.804233    100.0
feature        0.800000    0.84  0.819512    100.0
question       0.754717    0.80  0.776699    100.0
accuracy       0.800000    0.80  0.800000      0.8
macro avg      0.802883    0.80  0.800148    300.0
weighted avg   0.802883    0.80  0.800148    300.0


## Bitcoin Repo Testing

In [ ]:
y_true_bc = []
y_pred_bc = []

iterations = len(test_data_bitcoin)

# Now let's loop through the test data and classify the GitHub pull requests
for i in range(iterations):
    correct_label = test_data_bitcoin.iloc[i]['label'].lower()
    description = f"{test_data_bitcoin.iloc[i]['title']}\n{test_data_bitcoin.iloc[i]['body']}"
    print(f"Correct PR type: {correct_label}")
    
    prompt = f"Classify, IN ONLY 1 WORD, the following GitHub pull request as 'feature', 'bug', or 'question' based on its title and body:\n{description}"
    response = query_chatgpt(prompt, bitcoin_ft_model)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = re.sub(r'[^A-Za-z]+', '', response).lower().strip()
    print(f"Predicted PR type: {predicted_label}")
    
    # Append to lists for evaluation
    y_true_bc.append(correct_label)
    y_pred_bc.append(predicted_label)
    time.sleep(6)  # Wait for 6 seconds before retrying

# See output on outputs/cell68output.txt

In [ ]:
# Calculate weighted average F1-score, precision, and recall
f1_bc = f1_score(y_true_bc, y_pred_bc, labels=labels, average='weighted')
precision_bc = precision_score(y_true_bc, y_pred_bc, labels=labels, average='weighted')
recall_bc = recall_score(y_true_bc, y_pred_bc, labels=labels, average='weighted')

# Calculate confusion matrix
cm_bc = confusion_matrix(y_true_bc, y_pred_bc, labels=labels)

cm_df_bc = pd.DataFrame(cm_bc, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results_bc = {}
for i, label in enumerate(labels):
    results_bc[label] = {'TP': cm_bc[i, i]}
    results_bc[label]['FP'] = cm_bc[:, i].sum() - cm_bc[i, i]
    results_bc[label]['FN'] = cm_bc[i, :].sum() - cm_bc[i, i]
    results_bc[label]['TN'] = cm_bc.sum() - (results_bc[label]['TP'] + results_bc[label]['FP'] + results_bc[label]['FN'])

# Print results_bc
for label, metrics in results_bc.items():
    print(f"{label}: {metrics}")

# Save results_bc to CSV
results_bc_df = pd.DataFrame(results_bc).T
results_bc_df['F1-score'] = f1_bc
results_bc_df['Recall'] = recall_bc
results_bc_df['Precision'] = precision_bc

results_bc_df.to_csv('metrics/confusion_matrix_bc.csv')

print(f"Precision = {precision_bc}")
print(f"Recall = {recall_bc}")
print(f"F1-score = {f1_bc}")

feature: {'TP': 89, 'FP': 18, 'FN': 11, 'TN': 182}
bug: {'TP': 80, 'FP': 29, 'FN': 20, 'TN': 171}
question: {'TP': 62, 'FP': 22, 'FN': 38, 'TN': 178}
Precision = 0.7679386310527527
Recall = 0.77
F1-score = 0.7664555547850743


### F1-Score: 76.65%

In [ ]:
# Create a classification report
report_bc = classification_report(y_true_bc, y_pred_bc, labels=['bug', 'feature', 'question'], target_names=['bug', 'feature', 'question'], zero_division=0, output_dict=True)

# Convert the report to a DataFrame
report_df_bc = pd.DataFrame(report_bc).transpose()

# Print the classification report
print(report_df_bc)

              precision  recall  f1-score  support
bug            0.733945    0.80  0.765550   100.00
feature        0.831776    0.89  0.859903   100.00
question       0.738095    0.62  0.673913   100.00
accuracy       0.770000    0.77  0.770000     0.77
macro avg      0.767939    0.77  0.766456   300.00
weighted avg   0.767939    0.77  0.766456   300.00


## OpenCV Repo Testing

In [ ]:
y_true_oc = []
y_pred_oc = []

iterations = len(test_data_opencv)

# Now let's loop through the test data and classify the GitHub pull requests
for i in range(iterations):
    correct_label = test_data_opencv.iloc[i]['label'].lower()
    description = f"{test_data_opencv.iloc[i]['title']}\n{test_data_opencv.iloc[i]['body']}"
    print(f"Correct PR type: {correct_label}")
    
    prompt = f"Classify, IN ONLY 1 WORD, the following GitHub pull request as 'feature', 'bug', or 'question' based on its title and body:\n{description}"
    response = query_chatgpt(prompt, opencv_ft_model)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = re.sub(r'[^A-Za-z]+', '', response).lower().strip()
    print(f"Predicted PR type: {predicted_label}")
    
    # Append to lists for evaluation
    y_true_oc.append(correct_label)
    y_pred_oc.append(predicted_label)
    time.sleep(6)  # Wait for 6 seconds before retrying

# See output on outputs/cell73output.txt

In [ ]:
# Calculate weighted average F1-score, precision, and recall
f1_oc = f1_score(y_true_oc, y_pred_oc, labels=labels, average='weighted')
precision_oc = precision_score(y_true_oc, y_pred_oc, labels=labels, average='weighted')
recall_oc = recall_score(y_true_oc, y_pred_oc, labels=labels, average='weighted')

# Calculate confusion matrix
cm_oc = confusion_matrix(y_true_oc, y_pred_oc, labels=labels)

cm_df_oc = pd.DataFrame(cm_oc, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results_oc = {}
for i, label in enumerate(labels):
    results_oc[label] = {'TP': cm_oc[i, i]}
    results_oc[label]['FP'] = cm_oc[:, i].sum() - cm_oc[i, i]
    results_oc[label]['FN'] = cm_oc[i, :].sum() - cm_oc[i, i]
    results_oc[label]['TN'] = cm_oc.sum() - (results_oc[label]['TP'] + results_oc[label]['FP'] + results_oc[label]['FN'])

# Print results_oc
for label, metrics in results_oc.items():
    print(f"{label}: {metrics}")

# Save results_oc to CSV
results_oc_df = pd.DataFrame(results_oc).T
results_oc_df['F1-score'] = f1_oc
results_oc_df['Recall'] = recall_oc
results_oc_df['Precision'] = precision_oc

results_oc_df.to_csv('metrics/confusion_matrix_oc.csv')

print(f"Precision = {precision_oc}")
print(f"Recall = {recall_oc}")
print(f"F1-score = {f1_oc}")

feature: {'TP': 80, 'FP': 14, 'FN': 20, 'TN': 186}
bug: {'TP': 82, 'FP': 28, 'FN': 18, 'TN': 172}
question: {'TP': 81, 'FP': 15, 'FN': 19, 'TN': 185}
Precision = 0.8134227917472598
Recall = 0.81
F1-score = 0.8107417537461722


### F1-score: 81.07%


In [ ]:
# Create a classification report
report_oc = classification_report(y_true_oc, y_pred_oc, labels=['bug', 'feature', 'question'], target_names=['bug', 'feature', 'question'], zero_division=0, output_dict=True)

# Convert the report to a DataFrame
report_df_oc = pd.DataFrame(report_oc).transpose()

# Print the classification report
print(report_df_oc)

              precision  recall  f1-score  support
bug            0.745455    0.82  0.780952   100.00
feature        0.851064    0.80  0.824742   100.00
question       0.843750    0.81  0.826531   100.00
accuracy       0.810000    0.81  0.810000     0.81
macro avg      0.813423    0.81  0.810742   300.00
weighted avg   0.813423    0.81  0.810742   300.00


## Initial results  
With results gathered for all repositories tested against their respective trained models, we're poised to consolidate the confusion matrix data and derive the overall metrics.

In [2]:
def calculate_average(values):
    total = 0
    for value in values:
        total += value
    return total/len(values)

### BUG ###
facebook_bug_precision = 0.8333
tensorflow_bug_precision = 0.9167 
microsoft_bug_precision = 0.8539 
bitcoin_bug_precision = 0.7339 
opencv_bug_precision = 0.7455 
bug_precision_values = [facebook_bug_precision, tensorflow_bug_precision, microsoft_bug_precision, bitcoin_bug_precision, opencv_bug_precision]

facebook_bug_recall = 0.9500
tensorflow_bug_recall = 0.88
microsoft_bug_recall = 0.76
bitcoin_bug_recall = 0.8000
opencv_bug_recall = 0.8200 
bug_recall_values = [facebook_bug_recall, tensorflow_bug_recall, microsoft_bug_recall, bitcoin_bug_recall, opencv_bug_recall]

facebook_bug_f1 = 0.8878
tensorflow_bug_f1 = 0.8980
microsoft_bug_f1 = 0.8042
bitcoin_bug_f1 = 0.7656
opencv_bug_f1 = 0.7810
bug_f1_values = [facebook_bug_f1, tensorflow_bug_f1, microsoft_bug_f1, bitcoin_bug_f1, opencv_bug_f1]


### FEATURE ###
facebook_feature_precision = 0.8557
tensorflow_feature_precision = 0.9310
microsoft_feature_precision = 0.80
bitcoin_feature_precision = 0.8318
opencv_feature_precision = 0.8511
feature_precision_values = [facebook_feature_precision, tensorflow_feature_precision, microsoft_feature_precision, bitcoin_feature_precision, opencv_feature_precision]

facebook_feature_recall = 0.8900
tensorflow_feature_recall = 0.81
microsoft_feature_recall = 0.84
bitcoin_feature_recall = 0.8900
opencv_feature_recall = 0.8000
feature_recall_values = [facebook_feature_recall, tensorflow_feature_recall, microsoft_feature_recall, bitcoin_feature_recall, opencv_feature_recall]

facebook_feature_f1 = 0.8725
tensorflow_feature_f1 = 0.8663
microsoft_feature_f1 = 0.8195
bitcoin_feature_f1 = 0.8599
opencv_feature_f1 = 0.8247
feature_f1_values = [facebook_feature_f1, tensorflow_feature_f1, microsoft_feature_f1, bitcoin_feature_f1, opencv_feature_f1]


### QUESTION ###
facebook_question_precision = 0.9024
tensorflow_question_precision = 0.7607
microsoft_question_precision = 0.7547
bitcoin_question_precision = 0.7381
opencv_question_precision = 0.8438
question_precision_values = [facebook_question_precision, tensorflow_question_precision, microsoft_question_precision, bitcoin_question_precision, opencv_question_precision]

facebook_question_recall = 0.7400
tensorflow_question_recall = 0.89
microsoft_question_recall = 0.80
bitcoin_question_recall = 0.6200
opencv_question_recall = 0.8100
question_recall_values = [facebook_question_recall, tensorflow_question_recall, microsoft_question_recall, bitcoin_question_recall, opencv_question_recall]

facebook_question_f1 = 0.8132
tensorflow_question_f1 = 0.8203
microsoft_question_f1 = 0.7767
bitcoin_question_f1 = 0.6739
opencv_question_f1 = 0.8265
question_f1_values = [facebook_question_f1, tensorflow_question_f1, microsoft_question_f1, bitcoin_question_f1, opencv_question_f1]


### AVERAGE ###
facebook_average_precision = 0.8635
tensorflow_average_precision = 0.8695
microsoft_average_precision = 0.8029
bitcoin_average_precision = 0.7679
opencv_average_precision = 0.8134
average_precision_values = [facebook_average_precision, tensorflow_average_precision, microsoft_average_precision, bitcoin_average_precision, opencv_average_precision]

facebook_average_recall = 0.8600
tensorflow_average_recall = 0.86
microsoft_average_recall = 0.8
bitcoin_average_recall = 0.7700
opencv_average_recall = 0.8100
average_recall_values = [facebook_average_recall, tensorflow_average_recall, microsoft_average_recall, bitcoin_average_recall, opencv_average_recall]

facebook_average_f1 = 0.8579
tensorflow_average_f1 = 0.8615
microsoft_average_f1 = 0.8001
bitcoin_average_f1 = 0.7665
opencv_average_f1 = 0.8107
average_f1_values = [facebook_average_f1, tensorflow_average_f1, microsoft_average_f1, bitcoin_average_f1, opencv_average_f1]

# Calculating overall (average) values  
overall_bug_f1 = calculate_average(bug_f1_values)
overall_bug_precision = calculate_average(bug_precision_values)
overall_bug_recall = calculate_average(bug_recall_values)
overall_feature_f1 = calculate_average(feature_f1_values)
overall_feature_precision = calculate_average(feature_precision_values)
overall_feature_recall = calculate_average(feature_recall_values)
overall_question_f1 = calculate_average(question_f1_values)
overall_question_precision = calculate_average(question_precision_values)
overall_question_recall = calculate_average(question_recall_values)
overall_average_f1 = calculate_average(average_f1_values)
overall_average_precision = calculate_average(average_precision_values)
overall_average_recall = calculate_average(average_recall_values)

print("Overall Results: ")
# Formatting the results
formatted_metrics = {
    "Bug": {
        "Precision": overall_bug_precision, 
        "Recall": overall_bug_recall, 
        "F1-Score": overall_bug_f1
    },
    "Feature": {
        "Precision": overall_feature_precision, 
        "Recall": overall_feature_recall, 
        "F1-Score": overall_feature_f1
    },
    "Question": {
        "Precision": overall_question_precision, 
        "Recall": overall_question_recall, 
        "F1-Score": overall_question_f1
    },
    "Average": {
        "Precision": overall_average_precision, 
        "Recall": overall_average_recall, 
        "F1-Score": overall_average_f1
    }
}
formatted_metrics

Overall Results: 


{'Bug': {'Precision': 0.8166599999999999,
  'Recall': 0.842,
  'F1-Score': 0.8273199999999999},
 'Feature': {'Precision': 0.8539200000000001,
  'Recall': 0.8460000000000001,
  'F1-Score': 0.8485800000000001},
 'Question': {'Precision': 0.79994,
  'Recall': 0.772,
  'F1-Score': 0.7821200000000001},
 'Average': {'Precision': 0.82344,
  'Recall': 0.82,
  'F1-Score': 0.8193399999999998}}

## Data Cleaning: Method 2  
Upon analysis, opportunities for enhancement in our cleaning method surfaced, leading to the implementation of a new cleaning function.  
In the revised cleaning method (Method 2), emphasis was placed on stripping markdown text while adopting a strategy of replacing certain text elements to uphold the intended meaning.

In [6]:
# Function to convert Markdown to plain text
def strip_markdown(text):
    # Remove Markdown links
    text = re.sub(r'\[([^\]]*)\]\([^\)]*\)', r'\1', text)
    
    # Remove Markdown emphasis (* or _)
    text = re.sub(r'(\*|_)(.*?)\1', r'\2', text)
    
    # Remove Markdown inline code (`)
    text = re.sub(r'`([^`]+)`', r'\1', text)
    
    # Remove Markdown headers (##, ###, etc.)
    text = re.sub(r'#+\s*(.*?)\n', r'\1\n', text)
    
    # Remove other Markdown elements as needed
    
    return text

# Initialize counters for text cleaning
cleaned_count = 0
original_count = 0

def clean_text(text):
    global cleaned_count, original_count

    if not isinstance(text, str):
        original_count += 1
        return text

######################################
#        Standardize The Text        #
######################################

    # Lowercasing should be one of the first steps to ensure uniformity
    text = text.lower()

######################################
#         Remove Characters          #
######################################

    # Remove emojis, special characters, and punctuation
    text = emoji.demojize(text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)

######################################
#         Remove/Replace Text        #
######################################

    # Remove specific phrases "Website or app" and "local react development"
    text = text.replace("website or app", "")
    text = text.replace("local react development", "")

    # Replace URLs, HTML tags, user mentions, and markdown image references
    text = re.sub(r'https?://\S+|www\.\S+', '<URL>', text)
    text = re.sub(r'<.*?>', '<HTML_TAG>', text)
    text = re.sub(r'@\w+', '<USER>', text)
    text = re.sub(r'!\[image\]\(.*?\)', '<IMAGE>', text)

    # Remove text starting with "DevTools" and ending with "(automated)"
    text = re.sub(r'DevTools.*?\(automated\)', '', text)



        # Strip markdown formatting
    text = strip_markdown(text)

######################################
#        Tidy Up Whitespaces         #
######################################

    # Remove consecutive whitespaces and replace with a single space
    text = re.sub(r'\s+', ' ', text)

######################################
#            Final Things            #
######################################

    # Tokenize the text into words
    words = text.split()

    # Remove words that are over 20 characters
    words = [word for word in words if len(word) <= 20]

    # Join the remaining words back into cleaned text
    cleaned_text = ' '.join(words)

    cleaned_count += 1
    return cleaned_text

# Applying clean_text function to test and train data
test_data['body'] = test_data['body'].apply(clean_text)
test_data['title'] = test_data['title'].apply(clean_text)

train_data['body'] = train_data['body'].apply(clean_text)
train_data['title'] = train_data['title'].apply(clean_text)

# Displaying cleaning statistics
print(f"Cleaned {cleaned_count} times.")
print(f"Returned original text {original_count} times.")

Cleaned 5998 times.
Returned original text 2 times.


In [7]:
test_data_facebook = test_data[: 300]
test_data_tensorflow = test_data[300: 600]
test_data_microsoft = test_data[600: 900]
test_data_bitcoin = test_data[900: 1200]
test_data_opencv= test_data[1200: 1500]

In [9]:
train_data_facebook = train_data[: 300]
train_data_tensorflow = train_data[300: 600]
train_data_microsoft = train_data[600: 900]
train_data_bitcoin = train_data[900: 1200]
train_data_opencv= train_data[1200: 1500]

## Improved models  
Upon analyzing the step metrics of the fine-tuned models, it became evident that certain models, specifically those associated with the TensorFlow, Microsoft, and OpenCV repositories, exhibited training_loss figures indicating potential for improvement.

Considering this insight, we opted to develop new fine-tuned models, augmenting the epochs and integrating the enhanced cleaning method for these specific repositories.

## Tensorflow Improved model
For this tensorflow improved model we utilized the cleaning method 2 and 10 epochs on the fine-tuning processes. All the initial models displayed above on this notebook used cleaning method 1 and 3 epochs on the fine-tunning process.

In [ ]:
import tiktoken

max_content_tokens = 3999
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Function to truncate the message and avoid passing the limit of 4k tokens per gpt-3.5 fine-tuned model limitations
def truncate_message(message, max_length):
    tokens = encoding.encode(message)
    if len(tokens) > max_length:
        truncated_tokens = tokens[:max_length]
        message = encoding.decode(truncated_tokens)
    return message

# Open the file in write mode
with open('data/conversationaldata/conversational_data_tensorflow_new.jsonl', 'w', encoding='utf-8') as f:
    # Iterate over the rows in the DataFrame
    for index, row in train_data_tensorflow.iterrows():
        # Create the user message by formatting the prompt with the title and body
        user_message = f"Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:\n{row['title']}\n{row['body']}"
        
        # Truncate the prompt if necessary
        user_message = truncate_message(user_message, max_content_tokens)
        # Create the assistant message by taking the label
        assistant_message = row['label']
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "GitHub Issue Report Classifier"},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

In [ ]:
ft_file_tf_new = client.files.create(
  file=open("data/conversationaldata/conversational_data_tensorflow_new.jsonl", "rb"),
  purpose="fine-tune"
)
ft_file_tf_new.id

FileObject(id='file-BJCK5M6m756cgb0G6peUXb91', bytes=547089, created_at=1701744292, filename='conversational_data_tensorflow_new.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
## Creating a fine-tuned model
ft_job_tf_new = client.fine_tuning.jobs.create(
  training_file=ft_file_tf_new.id, 
  model="gpt-3.5-turbo",
  suffix= "tf-issueclassifier",
  hyperparameters={"n_epochs": 10}
)
ft_job_tf_new.id

FineTuningJob(id='ftjob-AbcuR8vh5M8kQDIV8hhLoXpe', created_at=1701744304, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-CpaRU3Zq9ePCCtbhezmcbgrg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-BJCK5M6m756cgb0G6peUXb91', validation_file=None)

In [ ]:
# Retrieving the state of a fine-tune
tensorflow_ft_model_new = client.fine_tuning.jobs.retrieve(ft_job_tf_new.id).fine_tuned_model
print(tensorflow_ft_model_new)

ft:gpt-3.5-turbo-0613:gcucst440:tf-issueclassifier:8SGhlOsl


In [ ]:
# You can track the progress of your fine-tuning job by listing the lastest events. On our models it took about 3 hours to fine-tune each model
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_tf_new.id, limit=20)

In [ ]:
y_true_tf_new = []
y_pred_tf_new = []

iterations = len(test_data_tensorflow)

# Now let's loop through the test data and classify the GitHub pull requests
for i in range(iterations):
    correct_label = test_data_tensorflow.iloc[i]['label'].lower()
    description = f"{test_data_tensorflow.iloc[i]['title']} \n {test_data_tensorflow.iloc[i]['body']}"
    print(f"Correct PR type: {correct_label}")
    
    prompt = f"Classify, IN ONLY 1 WORD, the following GitHub pull request as 'feature', 'bug', or 'question' based on its title and body:\n{description}"
    response = query_chatgpt(prompt, tensorflow_ft_model_new)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = re.sub(r'[^A-Za-z]+', '', response).lower().strip()
    print(f"Predicted PR type: {predicted_label}")
    
    # Append to lists for evaluation
    y_true_tf_new.append(correct_label)
    y_pred_tf_new.append(predicted_label)
    time.sleep(6)  # Wait for 6 seconds before retrying

# See output on outputs/cell89output.txt

In [52]:
# Calculate weighted average F1-score, precision, and recall
f1_tf_new = f1_score(y_true_tf_new, y_pred_tf_new, labels=labels, average='weighted')
precision_tf_new = precision_score(y_true_tf_new, y_pred_tf_new, labels=labels, average='weighted')
recall_tf_new = recall_score(y_true_tf_new, y_pred_tf_new, labels=labels, average='weighted')

# Calculate confusion matrix
cm_tf_new = confusion_matrix(y_true_tf_new, y_pred_tf_new, labels=labels)

cm_df_tf_new = pd.DataFrame(cm_tf_new, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results_tf_new = {}
for i, label in enumerate(labels):
    results_tf_new[label] = {'TP': cm_tf_new[i, i]}
    results_tf_new[label]['FP'] = cm_tf_new[:, i].sum() - cm_tf_new[i, i]
    results_tf_new[label]['FN'] = cm_tf_new[i, :].sum() - cm_tf_new[i, i]
    results_tf_new[label]['TN'] = cm_tf_new.sum() - (results_tf_new[label]['TP'] + results_tf_new[label]['FP'] + results_tf_new[label]['FN'])

# Print results_tf_new
for label, metrics in results_tf_new.items():
    print(f"{label}: {metrics}")

# Save results_tf_new to CSV
results_tf_new_df = pd.DataFrame(results_tf_new).T
results_tf_new_df['F1-score'] = f1_tf_new
results_tf_new_df['Recall'] = recall_tf_new
results_tf_new_df['Precision'] = precision_tf_new

results_tf_new_df.to_csv('metrics/confusion_matrix_tf_new.csv')

print(f"Precision = {precision_tf_new}")
print(f"Recall = {recall_tf_new}")
print(f"F1-score = {f1_tf_new}")

feature: {'TP': 82, 'FP': 6, 'FN': 18, 'TN': 194}
bug: {'TP': 88, 'FP': 9, 'FN': 12, 'TN': 191}
question: {'TP': 91, 'FP': 24, 'FN': 9, 'TN': 176}
Precision = 0.8767796748298766
Recall = 0.87
F1-score = 0.8707510228891061


In [53]:
# Create a classification report
report_tf_new = classification_report(y_true_tf_new, y_pred_tf_new, labels=['bug', 'feature', 'question'], target_names=['bug', 'feature', 'question'], zero_division=0, output_dict=True)

# Convert the report to a DataFrame
report_df_tf_new = pd.DataFrame(report_tf_new).transpose()

# Print the classification report
print(report_df_tf_new)

              precision  recall  f1-score  support
bug            0.907216    0.88  0.893401   100.00
feature        0.931818    0.82  0.872340   100.00
question       0.791304    0.91  0.846512   100.00
accuracy       0.870000    0.87  0.870000     0.87
macro avg      0.876780    0.87  0.870751   300.00
weighted avg   0.876780    0.87  0.870751   300.00


## OpenCV Improved Model
Cleaning method 2 and 6 epochs on fine-tuning process.

Employing a new cleaning method alongside 6 epochs, we chose this iteration as our experimentation with 10 epochs indicated it to be excessive for our TensorFlow model.

In [58]:
# Open the file in write mode
with open('data/conversationaldata/conversational_data_opencv_new.jsonl', 'w', encoding='utf-8') as f:
    # Iterate over the rows in the DataFrame
    for index, row in train_data_opencv.iterrows():
        # Create the user message by formatting the prompt with the title and body
        user_message = f"Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:\n{row['title']}\n{row['body']}"
        
        # Truncate the prompt if necessary
        user_message = truncate_message(user_message, max_content_tokens)
        # Create the assistant message by taking the label
        assistant_message = row['label']
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "GitHub Issue Report Classifier"},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

In [59]:
ft_file_oc_new = client.files.create(
  file=open("data/conversationaldata/conversational_data_opencv_new.jsonl", "rb"),
  purpose="fine-tune"
)
ft_file_oc_new.id

FileObject(id='file-BXCWSF32dn7asPO4Nfxqyhq5', bytes=605037, created_at=1701752486, filename='conversational_data_opencv_new.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [60]:
## Creating a fine-tuned model
ft_job_oc_new = client.fine_tuning.jobs.create(
  training_file=ft_file_oc_new.id, 
  model="gpt-3.5-turbo",
  suffix= "oc-issueclassifier",
  hyperparameters={"n_epochs": 6}
)
ft_job_oc_new.id

FineTuningJob(id='ftjob-rNIHMHUIy97BY1eDQDttqEZy', created_at=1701752539, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=6, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-CpaRU3Zq9ePCCtbhezmcbgrg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-BXCWSF32dn7asPO4Nfxqyhq5', validation_file=None)

In [88]:
# Retrieving the state of a fine-tune
opencv_ft_model_new = client.fine_tuning.jobs.retrieve(ft_job_oc_new.id).fine_tuned_model
print(opencv_ft_model_new)

ft:gpt-3.5-turbo-0613:gcucst440:oc-issueclassifier:8SJ2ph8V


In [ ]:
# You can track the progress of your fine-tuning job by listing the lastest events. On our models it took about 3 hours to fine-tune each model
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_oc_new.id, limit=20)

In [ ]:
y_true_oc_new = []
y_pred_oc_new = []

iterations = len(test_data_opencv)

# Now let's loop through the test data and classify the GitHub pull requests
for i in range(iterations):
    correct_label = test_data_opencv.iloc[i]['label'].lower()
    description = f"{test_data_opencv.iloc[i]['title']}\n{test_data_opencv.iloc[i]['body']}"
    print(f"Correct PR type: {correct_label}")
    
    prompt = f"Classify, IN ONLY 1 WORD, the following GitHub pull request as 'feature', 'bug', or 'question' based on its title and body:\n{description}"
    response = query_chatgpt(prompt, opencv_ft_model_new)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = re.sub(r'[^A-Za-z]+', '', response).lower().strip()
    print(f"Predicted PR type: {predicted_label}")
    
    # Append to lists for evaluation
    y_true_oc_new.append(correct_label)
    y_pred_oc_new.append(predicted_label)
    time.sleep(6)  # Wait for 6 seconds before retrying

# See output on outputs/cell98output.txt

In [74]:
# Calculate weighted average F1-score, precision, and recall
f1_oc_new = f1_score(y_true_oc_new, y_pred_oc_new, labels=labels, average='weighted')
precision_oc_new = precision_score(y_true_oc_new, y_pred_oc_new, labels=labels, average='weighted')
recall_oc_new = recall_score(y_true_oc_new, y_pred_oc_new, labels=labels, average='weighted')

# Calculate confusion matrix
cm_oc_new = confusion_matrix(y_true_oc_new, y_pred_oc_new, labels=labels)

cm_df_oc_new = pd.DataFrame(cm_oc_new, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results_oc_new = {}
for i, label in enumerate(labels):
    results_oc_new[label] = {'TP': cm_oc_new[i, i]}
    results_oc_new[label]['FP'] = cm_oc_new[:, i].sum() - cm_oc_new[i, i]
    results_oc_new[label]['FN'] = cm_oc_new[i, :].sum() - cm_oc_new[i, i]
    results_oc_new[label]['TN'] = cm_oc_new.sum() - (results_oc_new[label]['TP'] + results_oc_new[label]['FP'] + results_oc_new[label]['FN'])

# Print results_oc_new
for label, metrics in results_oc_new.items():
    print(f"{label}: {metrics}")

# Save results_oc_new to CSV
results_oc_new_df = pd.DataFrame(results_oc_new).T
results_oc_new_df['F1-score'] = f1_oc_new
results_oc_new_df['Recall'] = recall_oc_new
results_oc_new_df['Precision'] = precision_oc_new

results_oc_new_df.to_csv('metrics/confusion_matrix_oc_new.csv')

print(f"Precision = {precision_oc_new}")
print(f"Recall = {recall_oc_new}")
print(f"F1-score = {f1_oc_new}")

feature: {'TP': 80, 'FP': 8, 'FN': 20, 'TN': 192}
bug: {'TP': 86, 'FP': 32, 'FN': 14, 'TN': 168}
question: {'TP': 81, 'FP': 13, 'FN': 19, 'TN': 187}
Precision = 0.8332021986908391
Recall = 0.8233333333333334
F1-score = 0.8250354006223534


In [75]:
# Create a classification report
report_oc_new = classification_report(y_true_oc_new, y_pred_oc_new, labels=['bug', 'feature', 'question'], target_names=['bug', 'feature', 'question'], zero_division=0, output_dict=True)

# Convert the report to a DataFrame
report_df_oc_new = pd.DataFrame(report_oc_new).transpose()

# Print the classification report
print(report_df_oc_new)

              precision    recall  f1-score     support
bug            0.728814  0.860000  0.788991  100.000000
feature        0.909091  0.800000  0.851064  100.000000
question       0.861702  0.810000  0.835052  100.000000
accuracy       0.823333  0.823333  0.823333    0.823333
macro avg      0.833202  0.823333  0.825035  300.000000
weighted avg   0.833202  0.823333  0.825035  300.000000


## Microsoft Improved model
Cleaning Method 1 and 6 epochs

In [24]:
## Creating a fine-tuned model
ft_job_ms_new = client.fine_tuning.jobs.create(
  training_file=ft_file_ms, ## Using same file with old cleaning method
  model=microsoft_ft_model, ## Using old model as the base model
  suffix= "ms-issueclassifier",
  hyperparameters={"n_epochs": 3}
)
ft_job_ms_new

FineTuningJob(id='ftjob-ZRVLAfBawJPXM5OB06FPLSur', created_at=1701829890, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='ft:gpt-3.5-turbo-0613:gcucst440:ms-issueclassifier:8LLFl5QI', object='fine_tuning.job', organization_id='org-CpaRU3Zq9ePCCtbhezmcbgrg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-o2pr8FckpXuZhncbVF9Fykzl', validation_file=None)

In [31]:
# Retrieving the state of a fine-tune
microsoft_ft_model_new = client.fine_tuning.jobs.retrieve(ft_job_ms_new.id).fine_tuned_model
print(microsoft_ft_model_new)

ft:gpt-3.5-turbo-0613:gcucst440:ms-issueclassifier:8ScXRY4K


In [ ]:
# You can track the progress of your fine-tuning job by listing the lastest events. On our models it took about 3 hours to fine-tune each model
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_ms_new.id, limit=20)

In [ ]:
y_true_ms_new = []
y_pred_ms_new = []

iterations = len(test_data_microsoft)

# Now let's loop through the test data and classify the GitHub pull requests
for i in range(iterations):
    correct_label = test_data_microsoft.iloc[i]['label'].lower()
    description = f"{test_data_microsoft.iloc[i]['title']}\n{test_data_microsoft.iloc[i]['body']}"
    print(f"Correct PR type: {correct_label}")
    
    prompt = f"Classify, IN ONLY 1 WORD, the following GitHub pull request as 'feature', 'bug', or 'question' based on its title and body:\n{description}"
    response = query_chatgpt(prompt, microsoft_ft_model_new)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = re.sub(r'[^A-Za-z]+', '', response).lower().strip()
    print(f"Predicted PR type: {predicted_label}")
    
    # Append to lists for evaluation
    y_true_ms_new.append(correct_label)
    y_pred_ms_new.append(predicted_label)
    time.sleep(6)  # Wait for 6 seconds before retrying

# See output on outputs/cell105output.txt

In [34]:
# Calculate weighted average F1-score, precision, and recall
f1_ms_new = f1_score(y_true_ms_new, y_pred_ms_new, labels=labels, average='weighted')
precision_ms_new = precision_score(y_true_ms_new, y_pred_ms_new, labels=labels, average='weighted')
recall_ms_new = recall_score(y_true_ms_new, y_pred_ms_new, labels=labels, average='weighted')

# Calculate confusion matrix
cm_ms_new = confusion_matrix(y_true_ms_new, y_pred_ms_new, labels=labels)

cm_df_ms_new = pd.DataFrame(cm_ms_new, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results_ms_new = {}
for i, label in enumerate(labels):
    results_ms_new[label] = {'TP': cm_ms_new[i, i]}
    results_ms_new[label]['FP'] = cm_ms_new[:, i].sum() - cm_ms_new[i, i]
    results_ms_new[label]['FN'] = cm_ms_new[i, :].sum() - cm_ms_new[i, i]
    results_ms_new[label]['TN'] = cm_ms_new.sum() - (results_ms_new[label]['TP'] + results_ms_new[label]['FP'] + results_ms_new[label]['FN'])

# Print results_ms_new
for label, metrics in results_ms_new.items():
    print(f"{label}: {metrics}")

# Save results_ms_new to CSV
results_ms_new_df = pd.DataFrame(results_ms_new).T
results_ms_new_df['F1-score'] = f1_ms_new
results_ms_new_df['Recall'] = recall_ms_new
results_ms_new_df['Precision'] = precision_ms_new


results_ms_new_df.to_csv('metrics/confusion_matrix_ms_new.csv')

print(f"Precision = {precision_ms_new}")
print(f"Recall = {recall_ms_new}")
print(f"F1-score = {f1_ms_new}")

feature: {'TP': 87, 'FP': 20, 'FN': 13, 'TN': 180}
bug: {'TP': 80, 'FP': 14, 'FN': 20, 'TN': 186}
question: {'TP': 79, 'FP': 20, 'FN': 21, 'TN': 180}
Precision = 0.8207092466388549
Recall = 0.82
F1-score = 0.8197639424774652


In [35]:
# Create a classification report
report_ms_new = classification_report(y_true_ms_new, y_pred_ms_new, labels=['bug', 'feature', 'question'], target_names=['bug', 'feature', 'question'], zero_division=0, output_dict=True)

# Convert the report to a DataFrame
report_df_ms_new = pd.DataFrame(report_ms_new).transpose()

# Print the classification report
print(report_df_ms_new)

              precision  recall  f1-score  support
bug            0.851064    0.80  0.824742   100.00
feature        0.813084    0.87  0.840580   100.00
question       0.797980    0.79  0.793970   100.00
accuracy       0.820000    0.82  0.820000     0.82
macro avg      0.820709    0.82  0.819764   300.00
weighted avg   0.820709    0.82  0.819764   300.00


## Overall Results

To obtain the overall results we utilized the metrics from the regular facebook and bitcoin models, and the improved tensorflow, microsoft and opencv models. We then calculated the average of each metrics to have a complete overall metrics. 

In [1]:
def calculate_average(values):
    total = 0
    for value in values:
        total += value
    return total/len(values)

### BUG ###
facebook_bug_precision = 0.8333 ## Obtained in cell 56
tensorflow_bug_precision = 0.9072 ## Obtained in cell 91
microsoft_bug_precision = 0.8511 ## Obtained in cell 107
bitcoin_bug_precision = 0.7339 ## Obtained in cell 71
opencv_bug_precision = 0.7288 ## Obtained in cell 100
bug_precision_values = [facebook_bug_precision, tensorflow_bug_precision, microsoft_bug_precision, bitcoin_bug_precision, opencv_bug_precision]

facebook_bug_recall = 0.9500 ## Obtained in cell 56
tensorflow_bug_recall = 0.8800 ## Obtained in cell 91
microsoft_bug_recall = 0.8000 ## Obtained in cell 107
bitcoin_bug_recall = 0.8000 ## Obtained in cell 71
opencv_bug_recall = 0.8600 ## Obtained in cell 100
bug_recall_values = [facebook_bug_recall, tensorflow_bug_recall, microsoft_bug_recall, bitcoin_bug_recall, opencv_bug_recall]

facebook_bug_f1 = 0.8878 ## Obtained in cell 56
tensorflow_bug_f1 = 0.8934 ## Obtained in cell 91
microsoft_bug_f1 = 0.8247 ## Obtained in cell 107
bitcoin_bug_f1 = 0.7656 ## Obtained in cell 71
opencv_bug_f1 = 0.7890 ## Obtained in cell 100
bug_f1_values = [facebook_bug_f1, tensorflow_bug_f1, microsoft_bug_f1, bitcoin_bug_f1, opencv_bug_f1]


### FEATURE ###
facebook_feature_precision = 0.8557 ## Obtained in cell 56
tensorflow_feature_precision = 0.9318 ## Obtained in cell 91
microsoft_feature_precision = 0.8131## Obtained in cell 107
bitcoin_feature_precision = 0.8318 ## Obtained in cell 71
opencv_feature_precision = 0.9091 ## Obtained in cell 100
feature_precision_values = [facebook_feature_precision, tensorflow_feature_precision, microsoft_feature_precision, bitcoin_feature_precision, opencv_feature_precision]

facebook_feature_recall = 0.8900 ## Obtained in cell 56
tensorflow_feature_recall = 0.8200 ## Obtained in cell 91
microsoft_feature_recall = 0.8700 ## Obtained in cell 107
bitcoin_feature_recall = 0.8900 ## Obtained in cell 71
opencv_feature_recall = 0.8000 ## Obtained in cell 100
feature_recall_values = [facebook_feature_recall, tensorflow_feature_recall, microsoft_feature_recall, bitcoin_feature_recall, opencv_feature_recall]

facebook_feature_f1 = 0.8725 ## Obtained in cell 56
tensorflow_feature_f1 = 0.8723 ## Obtained in cell 91
microsoft_feature_f1 = 0.8406 ## Obtained in cell 107
bitcoin_feature_f1 = 0.8599 ## Obtained in cell 71
opencv_feature_f1 = 0.8511 ## Obtained in cell 100
feature_f1_values = [facebook_feature_f1, tensorflow_feature_f1, microsoft_feature_f1, bitcoin_feature_f1, opencv_feature_f1]


### QUESTION ###
facebook_question_precision = 0.9024 ## Obtained in cell 56
tensorflow_question_precision = 0.7913 ## Obtained in cell 91
microsoft_question_precision = 0.7980 ## Obtained in cell 107
bitcoin_question_precision = 0.7381 ## Obtained in cell 71
opencv_question_precision = 0.8617 ## Obtained in cell 100
question_precision_values = [facebook_question_precision, tensorflow_question_precision, microsoft_question_precision, bitcoin_question_precision, opencv_question_precision]

facebook_question_recall = 0.7400 ## Obtained in cell 56
tensorflow_question_recall = 0.9100 ## Obtained in cell 91
microsoft_question_recall = 0.7900 ## Obtained in cell 107
bitcoin_question_recall = 0.6200 ## Obtained in cell 71
opencv_question_recall = 0.8100 ## Obtained in cell 100
question_recall_values = [facebook_question_recall, tensorflow_question_recall, microsoft_question_recall, bitcoin_question_recall, opencv_question_recall]

facebook_question_f1 = 0.8132 ## Obtained in cell 56
tensorflow_question_f1 = 0.8465 ## Obtained in cell 91
microsoft_question_f1 = 0.7938 ## Obtained in cell 107
bitcoin_question_f1 = 0.6739 ## Obtained in cell 71
opencv_question_f1 = 0.8351 ## Obtained in cell 100
question_f1_values = [facebook_question_f1, tensorflow_question_f1, microsoft_question_f1, bitcoin_question_f1, opencv_question_f1]


### AVERAGE ###
facebook_average_precision = 0.8635 ## Obtained in cell 56
tensorflow_average_precision = 0.8768 ## Obtained in cell 91
microsoft_average_precision = 0.8207 ## Obtained in cell 107
bitcoin_average_precision = 0.7679 ## Obtained in cell 71 
opencv_average_precision = 0.8332 ## Obtained in cell 100
average_precision_values = [facebook_average_precision, tensorflow_average_precision, microsoft_average_precision, bitcoin_average_precision, opencv_average_precision]

facebook_average_recall = 0.8600 ## Obtained in cell 56
tensorflow_average_recall = 0.8700 ## Obtained in cell 91
microsoft_average_recall = 0.8200 ## Obtained in cell 107
bitcoin_average_recall = 0.7700 ## Obtained in cell 71
opencv_average_recall = 0.8233 ## Obtained in cell 100
average_recall_values = [facebook_average_recall, tensorflow_average_recall, microsoft_average_recall, bitcoin_average_recall, opencv_average_recall]

facebook_average_f1 = 0.8579 ## Obtained in cell 56
tensorflow_average_f1 = 0.8708 ## Obtained in cell 91
microsoft_average_f1 = 0.8198 ## Obtained in cell 107
bitcoin_average_f1 = 0.7665 ## Obtained in cell 71
opencv_average_f1 = 0.8250 ## Obtained in cell 100
average_f1_values = [facebook_average_f1, tensorflow_average_f1, microsoft_average_f1, bitcoin_average_f1, opencv_average_f1]

# Calculating overall (average) values  
overall_bug_f1 = calculate_average(bug_f1_values)
overall_bug_precision = calculate_average(bug_precision_values)
overall_bug_recall = calculate_average(bug_recall_values)
overall_feature_f1 = calculate_average(feature_f1_values)
overall_feature_precision = calculate_average(feature_precision_values)
overall_feature_recall = calculate_average(feature_recall_values)
overall_question_f1 = calculate_average(question_f1_values)
overall_question_precision = calculate_average(question_precision_values)
overall_question_recall = calculate_average(question_recall_values)
overall_average_f1 = calculate_average(average_f1_values)
overall_average_precision = calculate_average(average_precision_values)
overall_average_recall = calculate_average(average_recall_values)

print("Overall Results: ")
# Formatting the results
formatted_metrics = {
    "Bug": {
        "Precision": overall_bug_precision, 
        "Recall": overall_bug_recall, 
        "F1-Score": overall_bug_f1
    },
    "Feature": {
        "Precision": overall_feature_precision, 
        "Recall": overall_feature_recall, 
        "F1-Score": overall_feature_f1
    },
    "Question": {
        "Precision": overall_question_precision, 
        "Recall": overall_question_recall, 
        "F1-Score": overall_question_f1
    },
    "Average": {
        "Precision": overall_average_precision, 
        "Recall": overall_average_recall, 
        "F1-Score": overall_average_f1
    }
}
formatted_metrics

Overall Results: 


{'Bug': {'Precision': 0.8108599999999999, 'Recall': 0.858, 'F1-Score': 0.8321},
 'Feature': {'Precision': 0.8683,
  'Recall': 0.8540000000000001,
  'F1-Score': 0.85928},
 'Question': {'Precision': 0.8183,
  'Recall': 0.774,
  'F1-Score': 0.7925000000000001},
 'Average': {'Precision': 0.8324199999999999,
  'Recall': 0.82866,
  'F1-Score': 0.828}}